In [46]:
!pip install groq
import os
os.environ["GROQ_API_KEY"] = "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"
from groq import Groq

In [47]:
import csv
import json

In [48]:
# Load the JSON file
try:
    with open('data/Econet_data.json', 'r') as f:
        documents = json.load(f)
except FileNotFoundError:
    print("Error: The file 'data/Econet_data.json' was not found.")
    documents = []
except json.JSONDecodeError:
    print("Error: The file 'data/Econet_data.json' contains invalid JSON.")
    documents = []

if not documents:
    print("No documents loaded. Please check your data file and its location.")
else:
    print(f"Loaded {len(documents)} documents.")

Loaded 194 documents.


In [72]:
# Generate document IDs
def generate_document_id(doc):
    combined = f"{doc['Question'][:30]}-{doc['Answer'][:30]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [73]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [74]:
documents[3]

{'Question': 'I have WhatsApp bundle but am failing to make an App call?',
 'Answer': 'Please note that you cannot make calls using the WhatsApp bundle. In order to make calls, you will need Data bundles.',
 'id': 'be1f3e62'}

In [75]:
from collections import defaultdict

In [76]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [77]:
len(hashes), len(documents)

(172, 194)

In [78]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

8aeb90a4 2
2e8c06bb 2
55d73626 2
51eaa250 2
a34d8524 2
f9519900 2
b72301a3 2
bafc0cc3 2
462a0bfb 2
66cc5a80 2
4cacab7f 2
63e371e2 2
25f6fb23 2
9a5e402a 2
b51b04a9 2
4c686fe9 2
9f30bcc0 2
3ea69415 2
fe0e85bd 2
4d213c6c 2
6ebbc977 2
b2917423 2


In [79]:
hashes['4d213c6c']

[{'Question': 'Will I incur any additional charges on EcoSure?',
  'Answer': 'Normal EcoCash charges apply for all deductions made from your EcoCash wallet towards your EcoSure Personal Accident Cover premium payment.',
  'id': '4d213c6c'},
 {'Question': 'Will I incur any additional charges on EcoSure?',
  'Answer': 'Normal EcoCash charges apply for all deductions made from your EcoCash wallet towards your EcoSure Funeral Cover premium payment.',
  'id': '4d213c6c'}]

In [80]:
import json
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)




In [81]:
!head documents-with-ids.json

[
  {
    "Question": "I purchased a data bundle but it\u2019s not yet credited to my account.",
    "Answer": "Log your query on your self service platform, https://selfcare.econet.co.zw under \u201cMy Queries\u201d. Provide your full name and mobile number",
    "id": "a09c50b9"
  },
  {
    "Question": "Hi, I just purchased my daily bundles and they have exhausted though I haven\u2019t used them much.",
    "Answer": "We would like to inform you that all our bundles are usage based and you can now track your data, airtime or SMS usage via My Web self-care. You just need to follow this link: https://selfcare.econet.co.zw/ and register.",
    "id": "964d7a8b"


In [93]:
prompt_template = """
You are emulating a customer who is experiencing difficulties with our services. 
Formulate 5 questions a customer might ask based on a provided FAQ record.
The record contains answers to these questions,and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:


Question: {Question}
Answer: {Answer:}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [94]:
prompt = prompt_template.format(**doc)

In [95]:
doc= documents[2]

In [96]:
doc

{'Question': 'Can you send me data settings to my phone?',
 'Answer': 'Send an SMS “Device Type” followed by “Settings” for example, “Android Settings” to 222 to receive manual settings for phone',
 'id': '64f32973'}

In [97]:
print(prompt)

You are emulating a customer who is experiencing difficulties with our services. 
Formulate 5 questions a customer might ask based on a provided FAQ record.
The record contains answers to these questions,and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:


Question: Can you send me data settings to my phone?
Answer: Send an SMS “Device Type” followed by “Settings” for example, “Android Settings” to 222 to receive manual settings for phone

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]


In [98]:

    
client = Groq()
response = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )
    
json_response = response.choices[0].message.content
json_response

2024-09-25 12:04:17,048 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


'["How do I obtain data configuration for my mobile device", \n"Can you provide data settings for my mobile phone", \n"What is the process for getting data settings on my phone", \n"How do I manually set up internet on my phone using your service", \n"Is there a way to get internet settings sent directly to my phone"]'

In [99]:
json.loads(json_response)

['How do I obtain data configuration for my mobile device',
 'Can you provide data settings for my mobile phone',
 'What is the process for getting data settings on my phone',
 'How do I manually set up internet on my phone using your service',
 'Is there a way to get internet settings sent directly to my phone']

In [141]:
import hashlib
from tqdm.auto import tqdm
import pandas as pd
import os
from groq import Groq



# Function to generate questions using Groq
def generate_questions(doc):
    prompt = prompt_template.format(**doc)
    
    client = Groq()
    response = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )
    
    json_response = response.choices[0].message.content
    return json_response

In [142]:
from tqdm.auto import tqdm

In [143]:
results = {}

In [144]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

 16%|█████████████▉                                                                         | 31/194 [00:28<01:32,  1.75it/s]2024-09-25 12:47:42,752 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-09-25 12:47:42,753 - INFO - Retrying request to /openai/v1/chat/completions in 2.000000 seconds
2024-09-25 12:47:45,758 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
 16%|██████████████▎                                                                        | 32/194 [00:32<03:58,  1.47s/it]2024-09-25 12:47:45,941 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-09-25 12:47:45,942 - INFO - Retrying request to /openai/v1/chat/completions in 1.000000 seconds
2024-09-25 12:47:47,676 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
 17%|██████████████▊                          

In [165]:
parsed_results = {}
for doc_id, json_questions in results.items():
    try:
        parsed_results[doc_id] = json.loads(json_questions)
    except json.JSONDecodeError:
        print(f"Error parsing JSON for doc_id {doc_id}")
        parsed_results[doc_id] = []  # or however you want to handle errors

final_results = []
for doc_id, questions in parsed_results.items():
    if doc_id in doc_index:
        answer = doc_index[doc_id]['Answer']
        for q in questions:
            final_results.append((q, answer, doc_id))
    else:
        print(f"Warning: doc_id {doc_id} not found in doc_index")

df = pd.DataFrame(final_results, columns=['Question', 'Answer', 'id'])
print(f"Total rows in DataFrame: {len(df)}")
df.to_csv('ground-truth-data.csv', index=False)

Error parsing JSON for doc_id 964d7a8b
Error parsing JSON for doc_id 5a88df69
Error parsing JSON for doc_id 88ccd6d5
Error parsing JSON for doc_id 9844d785
Error parsing JSON for doc_id b0d701e7
Error parsing JSON for doc_id f0487f3c
Error parsing JSON for doc_id a041b137
Error parsing JSON for doc_id 692ae0aa
Error parsing JSON for doc_id b51b04a9
Error parsing JSON for doc_id 0f21a859
Error parsing JSON for doc_id 48e241bb
Error parsing JSON for doc_id d7cdd675
Total rows in DataFrame: 800


In [164]:
df

,Question,Answer,id
0,I bought a data bundle but it didn't show up o...,"Log your query on your self service platform, ...",a09c50b9
1,How do I report a problem with my data bundle ...,"Log your query on your self service platform, ...",a09c50b9
2,What should I do if my data bundle purchase di...,"Log your query on your self service platform, ...",a09c50b9
3,I purchased data and it hasn't reflected in my...,"Log your query on your self service platform, ...",a09c50b9
4,I did a data bundle purchase but I haven't rec...,"Log your query on your self service platform, ...",a09c50b9


In [ ]:
doc_index = {d['id']: d for d in documents}

In [152]:
import json

parsed_results = {}

# Loop through results and try to parse each entry
for doc_id, json_questions in results.items():
    try:
        # First, check if the value is already a list or dictionary (not a JSON string)
        if isinstance(json_questions, str):
            # Attempt to parse the JSON string
            parsed_results[doc_id] = json.loads(json_questions)
        else:
            # If it's not a string, it might already be a list or dict
            parsed_results[doc_id] = json_questions
    except json.JSONDecodeError as e:
        # Print the error and the specific doc_id causing it for troubleshooting
        print(f"Error parsing JSON for doc_id {doc_id}: {e}")
        print(f"Raw question data: {json_questions}")  # Show the problematic data for inspection

# Now you can proceed to use parsed_results as needed


Error parsing JSON for doc_id 964d7a8b: Expecting ',' delimiter: line 1 column 435 (char 434)
Raw question data: ["How can I determine if I've actually used up all my daily bundle allocations despite not using them frequently","What options are available for monitoring my daily bundle usage in real-time","Can I access my daily bundle usage information online","How do I keep track of my remaining daily bundle allocations to avoid unexpected depletion","Is there a web-based tool or platform where I can monitor and manage my daily bundle usage"
Error parsing JSON for doc_id 5a88df69: Expecting ',' delimiter: line 5 column 75 (char 390)
Raw question data: ["How can I retrieve airtime that was accidentally purchased for the wrong phone number", 
"Can I get a refund for airtime bought for a wrong number if it has not been used", 
"What happens to airtime bought for the wrong recipient number", 
"Is there any way to recover airtime purchased for an incorrect number", 
"Can I reverse airtime s

In [153]:
import json

parsed_results = {}

for doc_id, json_questions in results.items():
    try:
        # Clean up the JSON string (if needed) and parse it
        json_questions_cleaned = json_questions.replace('\n', '')  # Remove newlines
        
        # Attempt to load the cleaned JSON
        parsed_results[doc_id] = json.loads(json_questions_cleaned)
    except json.JSONDecodeError as e:
        # Print the error and the specific doc_id causing it for troubleshooting
        print(f"Error parsing JSON for doc_id {doc_id}: {e}")
        print(f"Raw question data: {json_questions}")  # Show the problematic data for inspection

        # Optionally, attempt to manually fix the JSON data
        # Here we could try to manually correct common issues if known
        # For example, using regular expressions to correct missing commas or other issues
        # However, manual inspection is often necessary for accurate fixes

# Now you can proceed to use parsed_results as needed


Error parsing JSON for doc_id 964d7a8b: Expecting ',' delimiter: line 1 column 435 (char 434)
Raw question data: ["How can I determine if I've actually used up all my daily bundle allocations despite not using them frequently","What options are available for monitoring my daily bundle usage in real-time","Can I access my daily bundle usage information online","How do I keep track of my remaining daily bundle allocations to avoid unexpected depletion","Is there a web-based tool or platform where I can monitor and manage my daily bundle usage"
Error parsing JSON for doc_id 5a88df69: Expecting ',' delimiter: line 1 column 387 (char 386)
Raw question data: ["How can I retrieve airtime that was accidentally purchased for the wrong phone number", 
"Can I get a refund for airtime bought for a wrong number if it has not been used", 
"What happens to airtime bought for the wrong recipient number", 
"Is there any way to recover airtime purchased for an incorrect number", 
"Can I reverse airtime 

In [151]:
resultsa

{'a09c50b9': '["I bought a data bundle but it didn\'t show up on my account yet, what should I do?", \n"How do I report a problem with my data bundle purchase?", \n"What should I do if my data bundle purchase didn\'t go through?", \n"I purchased data and it hasn\'t reflected in my account, how do I resolve this issue?", \n"I did a data bundle purchase but I haven\'t received it, what are my options?"]',
 '964d7a8b': '["How can I determine if I\'ve actually used up all my daily bundle allocations despite not using them frequently","What options are available for monitoring my daily bundle usage in real-time","Can I access my daily bundle usage information online","How do I keep track of my remaining daily bundle allocations to avoid unexpected depletion","Is there a web-based tool or platform where I can monitor and manage my daily bundle usage"',
 '64f32973': '["I\'m having trouble configuring my phone\'s internet settings, is there a way you can send me the data settings directly?", \

In [ ]:
final_results = []

for id, questions in parsed_resulst.items():
    Answer = doc_index[doc_id]['Answer']
    for q in questions:
        final_results.append((q,  Answer, id))

In [ ]:
df = pd.DataFrame(final_results, columns=['Question', 'Answer', 'id'])
df.to_csv('ground-truth-data.csv', index=False)

In [125]:
import pandas as pd

final_results = []

# Iterate over the results dictionary
for doc_id, questions in results.items():
    for q in questions:
        # Assuming each question is a dictionary with 'question' and 'answer' keys
        question = q.get('question')
        answer = q.get('answer')
        
        # Append the question, answer, and doc_id to final_results
        final_results.append((question, answer, doc_id))

# Create the DataFrame
df = pd.DataFrame(final_results, columns=['Question', 'Answer', 'id'])

# Save the DataFrame to a CSV file
df.to_csv('ground-truth-data.csv', index=False)

# Output DataFrame for verification
print(df.head())  # Display the first few rows to check the data


AttributeError: 'str' object has no attribute 'get'

In [116]:
for doc_id, questions in results.items():
    parsed_results[doc_id] = questions 

In [114]:
print(f"Generated questions for doc_id {doc_id}: {questions}")  # Check if questions are generated

Generated questions for doc_id bc6f0e68: ["What steps should I take when my airtime top-up is not processing", "I'm having trouble reloading airtime onto my mobile, what should I do", "My phone is failing to accept the recharge PIN, what can I do", "What can I do if my mobile recharge attempt keeps failing", "How do I resolve issues with topping up my phone airtime"]


In [118]:
# If the data in `results[doc_id]` is already a list of dictionaries, you don't need to parse it again
for doc_id, questions in results.items():
    parsed_results[doc_id] = questions  # No need for json.loads
print(parsed_results)

{'a09c50b9': '["I bought a data package, but my balance remains the same, what should I do", "I was supposed to get a data bundle on my phone, but I haven\'t received it, what should I do", "How do I get my data bundle activated", "After I bought data, I didn\'t see the changes on my phone, what can I do", "What steps should I take if my data bundle purchase wasn\'t successful"]', '964d7a8b': '["I\'ve just bought a daily bundle, but it seems like the data has been depleted quickly despite minimal usage.", \n"I recently purchased a daily bundle and the airtime was finished before I could use a substantial portion of it.",\n"I\'m experiencing issues with my daily bundle, specifically the data bundle, as it seems it\'s being depleted too quickly.",\n"I\'ve been noticing that my daily bundles are getting finished before I can use them to their full extent, can you explain why this might be happening.", \n"Can you explain to me why my daily bundle\'s data has run out even though I barely us

In [69]:
parsed_resulst = {}

for doc_id, json_questions in results.items():
    parsed_resulst[doc_id] = json.loads(json_questions)

In [70]:
doc_index = {d['id']: d for d in documents}

In [71]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

KeyError: 'course'

In [39]:


for doc in tqdm(documents, desc="Generating document IDs"):
    doc['id'] = generate_document_id(doc)

# Save documents with IDs
with open('data/documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)


Generating document IDs: 100%|█████████████████████████████████████████████████████████| 194/194 [00:00<00:00, 289468.15it/s]


In [41]:
import json
import hashlib
from tqdm.auto import tqdm
import pandas as pd
import os
from groq import Groq



# Function to generate questions using Groq
def generate_questions(doc):
    prompt = prompt_template.format(**doc)
    
    client = Groq()
    response = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )
    
    json_response = response.choices[0].message.content
    return json_response

# Generate questions for each document
results = {}
for doc in tqdm(documents, desc="Generating questions"):
    doc_id = doc['id']
    if doc_id in results:
        continue
    questions = generate_questions(doc)
    results[doc_id] = questions

# Save raw results
with open('raw_results.json', 'w') as f_out:
    json.dump(results, f_out, indent=2)

# Parse results with error handling
parsed_results = {}
for doc_id, json_questions in tqdm(results.items(), desc="Parsing results"):
    try:
        parsed_questions = json.loads(json_questions)
        if isinstance(parsed_questions, list):
            parsed_results[doc_id] = parsed_questions
        else:
            print(f"Warning: Unexpected format for doc_id {doc_id}. Expected a list, got {type(parsed_questions)}.")
            parsed_results[doc_id] = []
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON for doc_id {doc_id}: {str(e)}")
        print(f"Problematic JSON string: {json_questions}")
        parsed_results[doc_id] = []

# Create document index
doc_index = {d['id']: d for d in documents}



Parsing results: 100%|█████████████████████████████████████████████████████████████████| 172/172 [00:00<00:00, 185502.77it/s]

Error parsing JSON for doc_id d13e5bc4: Extra data: line 7 column 1 (char 496)
Problematic JSON string: ["I'm having trouble accessing my transactions history for the current month on EcoCash, can you help me?", 
"I need to verify my EcoCash transactions done this month, how can I view them?", 
"Where can I get a detailed breakdown of my EcoCash spending for this month?", 
"I'd like to download a statement of my EcoCash transactions for the past 30 days, can you guide me on how to do that?", 
"Can you please tell me how I can check my EcoCash account activity for this month on the website?"]

These questions are rephrased in different ways while still conveying the same query as the original question.
Error parsing JSON for doc_id b2fc8b43: Expecting ',' delimiter: line 1 column 141 (char 140)
Problematic JSON string: ["Are voice bundle minutes with Econet limited to calling numbers within the Econet network or can I also use them to call other networks?" "What happens when I call some

In [42]:
# Prepare final results
final_results = []
for doc_id, questions in tqdm(parsed_results.items(), desc="Preparing final results"):
    Question = doc_index[doc_id]['Question']
    Answer = doc_index[doc_id]['Answer']
    for q in questions:
        final_results.append((Question, Answer, doc_id))

# Create DataFrame and save to CSV
df = pd.DataFrame(final_results, columns=['Question','Answer', 'id'])
df.to_csv('data/ground-truth-data.csv', index=False)

print("Ground truth dataset has been created and saved to 'ground-truth-data.csv'")
print(f"Total documents processed: {len(documents)}")
print(f"Total documents with successfully parsed questions: {len(parsed_results)}")
print(f"Total generated questions: {len(final_results)}")

Preparing final results: 100%|█████████████████████████████████████████████████████████| 172/172 [00:00<00:00, 357492.71it/s]

Ground truth dataset has been created and saved to 'ground-truth-data.csv'
Total documents processed: 194
Total documents with successfully parsed questions: 172
Total generated questions: 835


In [44]:
 questions

["I'd like to know more about the funeral cover services offered by Econet, specifically what options are available and what they entail.",
 'How does the EcoSure product work in terms of funeral insurance and what benefits can I expect if I sign up for it?',
 'Can you explain to me what EcoSure Funeral Cover is and what kind of protection it offers in the event of a death?',
 "I'm considering purchasing funeral coverage, can you tell me more about the EcoSure product, including its features and what it covers?",
 "What is the EcoSure Funeral Cover policy, and how does it assist in the event of the policyholder's or their loved one's passing?"]

In [45]:
import json
import hashlib
from tqdm.auto import tqdm
import pandas as pd
import logging
from groq import Groq
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Groq API key
GROQ_API_KEY = os.getenv('GROQ_API_KEY')

# Prompt template
prompt_template = """
You are an AI assistant for a telecommunications company that also offers mobile banking and funeral cover (EcoSure) services. Generate 5 diverse questions a customer might ask based on the following FAQ entry. Ensure the questions are distinct and cover different aspects of the topic.

FAQ Entry:
Section: {section}
Question: {question}
Answer: {text}

Guidelines:
1. Use varying question structures (e.g., "What", "How", "Can you explain", "Is it possible to").
2. Include questions about specific features, processes, or common issues.
3. Consider potential clarifications or follow-ups a customer might need.
4. Avoid directly copying phrases from the FAQ; rephrase the content.
5. Ensure questions are relevant to customer support scenarios.

Provide the output as a JSON array of strings, without using code blocks:
["question1", "question2", "question3", "question4", "question5"]
"""

def load_documents(file_path):
    try:
        with open(file_path, 'r') as f:
            documents = json.load(f)
        logging.info(f"Loaded {len(documents)} documents from {file_path}")
        return documents
    except FileNotFoundError:
        logging.error(f"Error: The file '{file_path}' was not found.")
        return []
    except json.JSONDecodeError:
        logging.error(f"Error: The file '{file_path}' contains invalid JSON.")
        return []

def generate_document_id(doc):
    combined = f"{doc['Question'][:30]}-{doc['Answer'][:30]}"
    hash_object = hashlib.md5(combined.encode())
    return hash_object.hexdigest()[:8]

def generate_questions(doc, client):
    prompt = prompt_template.format(**doc)
    try:
        response = client.chat.completions.create(
            model="llama-3.1-70b-versatile",
            messages=[{"role": "user", "content": prompt}]
        )
        json_response = response.choices[0].message.content
        logging.info(f"Generated questions for doc_id {doc['id']}: {json_response}")
        return json_response
    except Exception as e:
        logging.error(f"Error generating questions for doc_id {doc['id']}: {str(e)}")
        return "[]"

def parse_and_validate_questions(doc_id, json_questions):
    try:
        parsed_questions = json.loads(json_questions)
        if isinstance(parsed_questions, list) and len(parsed_questions) == 5:
            if all(isinstance(q, str) for q in parsed_questions):
                return parsed_questions
            else:
                logging.warning(f"Invalid question format for doc_id {doc_id}. Expected all strings.")
        else:
            logging.warning(f"Unexpected format for doc_id {doc_id}. Expected a list of 5 questions.")
    except json.JSONDecodeError as e:
        logging.error(f"Error parsing JSON for doc_id {doc_id}: {str(e)}")
    return []

def deduplicate_questions(parsed_results):
    all_questions = set()
    deduplicated_results = {}
    for doc_id, questions in parsed_results.items():
        unique_questions = []
        for question in questions:
            normalized_question = question.lower().strip()
            if normalized_question not in all_questions:
                all_questions.add(normalized_question)
                unique_questions.append(question)
        if unique_questions:
            deduplicated_results[doc_id] = unique_questions
    return deduplicated_results

def main():
    # Load documents
    documents = load_documents('data/Econet_data.json')
    if not documents:
        logging.error("No documents loaded. Exiting.")
        return

    # Generate document IDs
    for doc in documents:
        doc['id'] = generate_document_id(doc)

    # Save documents with IDs
    with open('data/documents-with-ids.json', 'wt') as f_out:
        json.dump(documents, f_out, indent=2)

    # Initialize Groq client
    client = Groq(api_key=GROQ_API_KEY)

    # Generate questions
    results = {}
    for doc in tqdm(documents, desc="Generating questions"):
        doc_id = doc['id']
        if doc_id in results:
            logging.info(f"Skipping doc_id {doc_id} as it's already processed")
            continue
        questions = generate_questions(doc, client)
        results[doc_id] = questions

    # Save raw results
    with open('raw_results.json', 'w') as f_out:
        json.dump(results, f_out, indent=2)

    # Parse and validate results
    parsed_results = {}
    for doc_id, json_questions in tqdm(results.items(), desc="Parsing results"):
        parsed_questions = parse_and_validate_questions(doc_id, json_questions)
        if parsed_questions:
            parsed_results[doc_id] = parsed_questions
        else:
            logging.warning(f"Skipping doc_id {doc_id} due to parsing or validation issues")

    # Deduplicate questions
    deduplicated_results = deduplicate_questions(parsed_results)
    logging.info(f"Total documents after deduplication: {len(deduplicated_results)}")

    # Create document index
    doc_index = {d['id']: d for d in documents}

    # Prepare final results
    final_results = []
    for doc_id, questions in tqdm(deduplicated_results.items(), desc="Preparing final results"):
        original_question = doc_index[doc_id]['Question']
        answer = doc_index[doc_id]['Answer']
        for q in questions:
            final_results.append((original_question, q, answer, doc_id))

    # Create DataFrame and save to CSV
    df = pd.DataFrame(final_results, columns=['Original Question', 'Generated Question', 'Answer', 'id'])
    df.to_csv('data/ground-truth-data.csv', index=False)
    logging.info("Ground truth dataset has been created and saved to 'ground-truth-data.csv'")
    logging.info(f"Total documents processed: {len(documents)}")
    logging.info(f"Total documents with successfully parsed questions: {len(deduplicated_results)}")
    logging.info(f"Total generated questions: {len(final_results)}")

if __name__ == "__main__":
    main()

2024-09-25 11:00:16,700 - INFO - Loaded 194 documents from data/Econet_data.json
Generating questions:   0%|                                                                          | 0/194 [00:00<?, ?it/s]


KeyError: 'section'